In [1]:
import _pickle as cPickle
import pickle
import pandas as pd
import os
from scipy.spatial.distance import cdist
from sklearn.decomposition import PCA as sklearnPCA
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.manifold import MDS
from sklearn import cluster as Kcluster, metrics as Met ,preprocessing as preprocessing
import numpy as np

In [4]:
'''
#Create Sample Set . Holds the sample IDs and the labels under the column [accession,label]
SAMPLES = pd.read_csv(r"C:\Users\ajinkuriakose\Desktop\Computational Biology\Project\p1_train.csv", sep=',', error_bad_lines=False)
labels = ['CEU','FIN','GBR','TSI','YRI']

sample_set = pd.DataFrame()

# SAMPLES_FILETERED = SAMPLES.query("label in @labels").sample(n=10)['accession']
# print(SAMPLES_FILETERED)
for lab in labels:
    SAMPLES_FILETERED = SAMPLES[SAMPLES.label == lab].sample(n=30)
    sample_set =sample_set.append(SAMPLES_FILETERED,ignore_index=False)
# print (SAMP['accession'])
# SAMP.to_csv('test.csv',index=False)
# pickle.dump(sample_set,open("sample75_ids.pkl",'wb'))
'''

In [5]:
'''
#Create Data Set
# sample_set= pickle.load(open("sample75_ids.pkl",'rb'))
sample_set_id = sample_set['accession']
# sample_set_label = sample_set['label']
# sample_set_id = ['ERR188021','ERR188022','ERR188023']
sample_data = pd.DataFrame()
root = r"C:\Users\ajinkuriakose\Desktop\Computational Biology\train"
for sampleid in sample_set_id:
    try:
        quantified_data = pd.read_csv(root+"\\"+sampleid+"\\bias\\quant.sf",sep='\t',error_bad_lines=False)
        quantified_data = pd.DataFrame(quantified_data.set_index('Name').stack()).transpose()
        sample_data = sample_data.append(quantified_data)
    except KeyboardInterrupt:
        break 
    pickle.dump(sample_data, open("data_150.pkl", 'wb'))
'''    

In [2]:
sample_data= pickle.load(open("data_75.pkl",'rb')) #data_150.pkl
sample_data= sample_data.dropna()

In [12]:
sample_data.shape
sample_data[0:10]

Name ENST00000456328.2                                      ENST00000450305.2  \
                Length EffectiveLength       TPM   NumReads            Length   
0               1657.0         1402.27  0.000000   0.000000             632.0   
0               1657.0         1518.06  0.162213   3.127660             632.0   
0               1657.0         1374.13  0.096966   5.372600             632.0   
0               1657.0         1503.66  0.000000   0.000000             632.0   
0               1657.0         1416.54  0.000809   0.017940             632.0   
0               1657.0         1405.11  0.000837   0.020518             632.0   
0               1657.0         1431.31  0.211032   7.465510             632.0   
0               1657.0         1389.36  0.797727  18.384100             632.0   
0               1657.0         1536.58  0.097932   2.988250             632.0   
0               1657.0         1421.20  0.000000   0.000000             632.0   

Name                               ENST00000488147.1                   ...     \
     EffectiveLength  TPM NumReads            Length EffectiveLength   ...      
0            474.727  0.0      0.0            1351.0        1223.110   ...      
0            520.851  0.0      0.0            1351.0        1195.490   ...      
0            469.183  0.0      0.0            1351.0        1250.460   ...      
0            498.230  0.0      0.0            1351.0        1018.440   ...      
0            422.938  0.0      0.0            1351.0         758.916   ...      
0            216.000  0.0      0.0            1351.0        1344.240   ...      
0            493.718  0.0      0.0            1351.0        1370.370   ...      
0            466.101  0.0      0.0            1351.0        1315.050   ...      
0            517.513  0.0      0.0            1351.0        1140.090   ...      
0            480.224  0.0      0.0            1351.0        1242.710   ...      

Name ENST00000361789.2          ENST00000387460.2                            \
                   TPM NumReads            Length EffectiveLength       TPM   
0              3475.87  47069.0              66.0            20.0   94.5189   
0              2839.06  31426.0              66.0            21.0  464.8980   
0              2889.02  94491.3              66.0            21.0  474.7570   
0              3059.78  43702.0              66.0            22.0  384.0920   
0              7061.75  99538.0              66.0            23.0  571.8870   
0              5844.33  85682.0              66.0            22.0  461.3730   
0              4681.41  95895.0              66.0            20.0  813.6040   
0              3629.21  49875.0              66.0            20.0  437.0840   
0              3919.78  67197.0              66.0            20.0  694.9330   
0              2493.67  54775.0              66.0            19.0  561.0230   

Name          ENST00000387461.2                                    
     NumReads            Length EffectiveLength      TPM NumReads  
0      32.000              68.0            20.0  2723.33   922.00  
0     124.000              68.0            21.0  2538.20   677.00  
0     402.000              68.0            21.0  2307.65  1954.00  
0     138.000              68.0            22.0  2148.69   772.00  
0     206.000              68.0            23.0  2889.97  1041.00  
0     177.141              68.0            23.0  2814.83  1129.86  
0     402.180              68.0            20.0  3703.72  1830.82  
0     145.000              68.0            21.0  2652.65   924.00  
0     276.000              68.0            21.0  3496.25  1458.00  
0     270.000              68.0            20.0  3420.89  1733.00  

[10 rows x 797296 columns]

In [14]:
def get_eigenvalues(data):
    U, V, W = np.linalg.svd(data, full_matrices=False)
    return U,V

def perf_pca(data):
    #U, V, W = np.linalg.svd(data, full_matrices=False)
    # print(U)
    #newV = [x for x in V if x >= 1]
    # find the number of principal components.
    # select the components with eigenvalue greater than 1.
    #dim = len(newV)
    sk_pca = sklearnPCA(n_components=1000)
    pca = sk_pca.fit_transform(data)
    return pca


# pca_sample = perf_pca(sample_data)

In [40]:
# Normalize the data
sample_norm_data = preprocessing.scale(np.array(sample_data))
# Compute the correlation matrix
sample_corr_data = np.corrcoef(sample_norm_data, rowvar=False)
pca_sample = perf_pca(sample_corr_data)

MemoryError: 

In [15]:
pca=perf_pca(sample_data)

In [16]:
pca.shape

(75, 75)